In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=42
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=NonInvasiveFetalECGThorax2(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (1800, 751)
first_test_shape= (1965, 751)
classes_quantity= 42
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
Count_labels= [36 52 45 40 48 44 36 50 46 56 36 39 46 42 40 39 52 45 40 49 43 40 35 41
 41 37 47 40 47 40 43 41 37 52 45 40 37 41 43 45 39 45]
max(train_feature_Altitude)= 42.0
min(train_feature_Altitude)= -5.4156817
first_train_sample=
 [ 3.3000000e+01  4.2513762e+00  4.2896489e+00  4.2280762e+00
  4.1292816e+00  3.9943007e+00  3.7654856e+00  3.4269719e+00
  3.0086621e+00  2.5496071e+00  2.0923628e+00  1.6631705e+00
  1.2298972e+00  7.4750542e-01  1.9019816e-01 -4.5405910e-01
 -1.1292452e+00 -1.7296598e+00 -2.2396899e+00 -2.7059539e+00
 -3.1589365e+00 -3.5353216e+00 -3.7954363e+00 -3.9314129e+00
 -3.9489070e+00 -3.9042555e+00 -3.8611114e+00 -3.8220483e+00
 -3.7560545e+00 -3.6858879e+00 -3.5917502e+00 -3.4210764e+00
 -3.1618318e+00 -2.893


final_train_shap= (1800, 481)


In [3]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (36, 481)
shape base_wn= (1800, 481)
shape furir_windws0= (1800, 480)
shape furir_windws= (1800, 481)
smpl_rte= 480
mx_wndws= 56


In [4]:
path=('{}\AugEvlV'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvlV/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [5]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [6]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],sclRaw(ecg,i,int(vlum_sig),mag=.2),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.05),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

shape and aug[0]= (481,) [ 2.96289062e+00  3.26562500e+00  3.52734375e+00  3.68359375e+00
  3.61132812e+00  3.35937500e+00  2.87109375e+00  2.35546875e+00
  1.97656250e+00  1.68359375e+00  1.12792969e+00  3.16162109e-01
 -2.95654297e-01 -6.28417969e-01 -8.75000000e-01 -1.09375000e+00
 -1.35644531e+00 -1.56347656e+00 -1.83593750e+00 -2.15234375e+00
 -2.28710938e+00 -2.19140625e+00 -1.92382812e+00 -1.60644531e+00
 -1.31542969e+00 -1.15039062e+00 -9.94140625e-01 -9.04296875e-01
 -8.74023438e-01 -8.60839844e-01 -8.22265625e-01 -8.04199219e-01
 -7.23632812e-01 -5.21484375e-01 -4.08447266e-01 -2.59521484e-01
 -8.34350586e-02 -2.97393799e-02 -7.52563477e-02 -1.24206543e-01
 -1.41723633e-01 -1.22741699e-01 -1.71630859e-01 -2.49511719e-01
 -3.51074219e-01 -3.49853516e-01 -2.08496094e-01 -7.84912109e-02
 -1.06445312e-01 -1.49902344e-01 -9.34448242e-02 -6.37817383e-02
 -8.88671875e-02 -5.00183105e-02  1.68132782e-03  1.56307220e-03
 -1.16806030e-02 -3.60717773e-02 -7.00073242e-02 -1.05529785e-01


class= 1 shape(new5)= (527, 486)	mx= 6.60546875	mn= -4.7734375

110 = 527


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


shape and aug[0]= (481,) [ 3.46573087e+00  3.69784709e+00  3.96529525e+00  4.00612814e+00
  3.74307133e+00  3.21159765e+00  2.56746925e+00  1.92994474e+00
  1.46598555e+00  9.64731192e-01  8.88691340e-02 -8.80416522e-01
 -1.39053043e+00 -1.78726031e+00 -2.16297199e+00 -2.45005653e+00
 -2.60974131e+00 -2.83516096e+00 -3.07869376e+00 -3.09150657e+00
 -2.88652022e+00 -2.60457938e+00 -2.24908467e+00 -1.85496610e+00
 -1.45844030e+00 -1.23740874e+00 -1.12486366e+00 -1.06574229e+00
 -1.00462494e+00 -9.83665796e-01 -9.28306122e-01 -7.93914691e-01
 -6.24159660e-01 -4.68343448e-01 -3.51447152e-01 -2.86977075e-01
 -2.13405333e-01 -1.83670507e-01 -2.34102268e-01 -2.85739841e-01
 -3.34528018e-01 -4.06499685e-01 -4.44599351e-01 -4.14590033e-01
 -3.60333957e-01 -3.34457026e-01 -2.96198897e-01 -2.76297126e-01
 -2.44749803e-01 -2.07776217e-01 -1.78506835e-01 -1.63446465e-01
 -1.17085306e-01 -1.13773713e-01 -1.48685511e-01 -1.77859040e-01
 -2.03180157e-01 -1.94744831e-01 -1.46619973e-01 -9.93531595e-02


class= 1 shape(new5)= (527, 486)	mx= 4.695675621993545	mn= -3.527230785150198

120 = 527
shape and aug[0]= (481,) [ 3.34785847e+00  3.65866924e+00  3.88299603e+00  3.64330267e+00
  2.92900582e+00  2.14906426e+00  1.64189083e+00  1.08750775e+00
  1.49338591e-02 -8.70104370e-01 -1.28575451e+00 -1.67121405e+00
 -2.06256667e+00 -2.30760459e+00 -2.58176749e+00 -2.76827616e+00
 -2.71245430e+00 -2.42289282e+00 -2.01884819e+00 -1.64623976e+00
 -1.40637874e+00 -1.25276088e+00 -1.13639965e+00 -1.02911901e+00
 -9.65491417e-01 -9.06441565e-01 -7.58210391e-01 -6.02071925e-01
 -4.21782808e-01 -2.66232200e-01 -1.64502584e-01 -1.06300742e-01
 -9.49770154e-02 -1.36552488e-01 -1.91008240e-01 -2.45318785e-01
 -2.79024102e-01 -2.94074682e-01 -3.03841782e-01 -2.69454625e-01
 -1.81036297e-01 -9.69747783e-02 -7.20555245e-02 -7.16637668e-02
 -5.01252759e-02 -4.17962753e-03  1.29159596e-03 -4.93940999e-02
 -1.12181848e-01 -1.32025152e-01 -9.60135409e-02 -7.51198713e-02
 -8.61374663e-02 -1.21502333e-01 -6.04455

...

class= 42 shape(new5)= (518, 486)	mx= 5.41796875	mn= -4.94140625

4210 = 518
shape and aug[0]= (481,) [ 2.98826172e+00  2.88570272e+00  2.65543116e+00  2.33329762e+00
  2.00606146e+00  1.60609986e+00  1.18671474e+00  7.38556344e-01
  1.81384305e-01 -4.41253390e-01 -9.34268299e-01 -1.33916010e+00
 -1.78116460e+00 -2.23563764e+00 -2.52862076e+00 -2.63473433e+00
 -2.72065198e+00 -2.82824836e+00 -2.78111981e+00 -2.65607577e+00
 -2.47219139e+00 -2.18558136e+00 -1.85895772e+00 -1.53792946e+00
 -1.27198114e+00 -1.04737836e+00 -8.55641326e-01 -7.03704101e-01
 -5.50633715e-01 -3.95288252e-01 -2.40212287e-01 -1.23487332e-01
 -4.52658382e-02 -3.99081059e-03  2.68556950e-02  4.09583796e-02
  1.13632304e-02 -2.57590414e-02 -8.96691100e-02 -1.37133138e-01
 -1.65066531e-01 -1.84523175e-01 -1.85490039e-01 -1.81074929e-01
 -1.54591914e-01 -1.29682708e-01 -7.95467050e-02 -3.11811320e-02
 -3.45220274e-03  1.79612204e-02  2.43074556e-02  3.45815203e-02
  2.90346693e-02  3.22830704e-02  2.03089520e-02  2.9

class= 42 shape(new5)= (518, 486)	mx= 3.690274217851797	mn= -3.4241096235720203

4220 = 518
shape and aug[0]= (481,) [ 3.04065021e+00  3.04110876e+00  2.84534773e+00  2.49808684e+00
  2.09323323e+00  1.64380690e+00  1.17315668e+00  6.56420190e-01
 -3.72691068e-02 -6.66845936e-01 -1.14812150e+00 -1.66982940e+00
 -2.22239884e+00 -2.58797704e+00 -2.75095182e+00 -2.88796910e+00
 -2.92938680e+00 -2.83598996e+00 -2.65982167e+00 -2.40618143e+00
 -2.07121252e+00 -1.72108139e+00 -1.39887295e+00 -1.11574052e+00
 -8.67255758e-01 -6.71027289e-01 -5.12452948e-01 -3.75182270e-01
 -2.18971550e-01 -7.91608687e-02  4.60184149e-03  5.40123168e-02
  7.02894872e-02  6.11750069e-02  1.58837840e-02 -4.07102185e-02
 -9.86440159e-02 -1.49024651e-01 -1.91734055e-01 -2.09417114e-01
 -1.83191107e-01 -1.50111292e-01 -1.09625630e-01 -6.66903105e-02
 -3.04590906e-02 -1.37507045e-02  1.74490607e-03  1.99022395e-02
  2.93394216e-02  3.09016470e-02  2.65242321e-02  1.97502304e-02
  7.05841513e-04 -9.81943261e-03 -3.37

class= 42 shape(new5)= (518, 486)	mx= 3.342697679038266	mn= -3.137715696631408

4230 = 518
shape and aug[0]= (481,) [ 3.20956733e+00  3.12713450e+00  2.87060332e+00  2.49252759e+00
  2.02707591e+00  1.56939415e+00  1.15714277e+00  7.01604359e-01
  1.08748598e-01 -6.54511029e-01 -1.27595773e+00 -1.66071731e+00
 -2.00627046e+00 -2.29008497e+00 -2.44574760e+00 -2.48693487e+00
 -2.59612568e+00 -2.77114539e+00 -2.77968371e+00 -2.63888775e+00
 -2.46034351e+00 -2.18701557e+00 -1.86600415e+00 -1.54911361e+00
 -1.27568272e+00 -1.06357151e+00 -8.84182673e-01 -7.52093822e-01
 -6.57334866e-01 -5.52185702e-01 -4.20212767e-01 -3.01348730e-01
 -2.06372577e-01 -8.74642433e-02  9.29053924e-03  5.02991881e-02
  5.62363784e-02  4.83558710e-02  1.45715060e-02 -3.89684459e-02
 -8.55032656e-02 -1.32933535e-01 -1.54316983e-01 -1.58691870e-01
 -1.32922920e-01 -1.02264535e-01 -5.66434555e-02 -1.75457995e-03
  3.18143297e-02  7.30788566e-02  1.02238971e-01  1.21267296e-01
  1.01253518e-01  7.87218355e-02  7.043

class= 42 shape(new5)= (518, 486)	mx= 3.3925710641829396	mn= -3.1440935610303558

4240 = 518
shape and aug[0]= (481,) [ 3.00781250e+00  3.10546875e+00  3.06054688e+00  2.90820312e+00
  2.59179688e+00  2.23437500e+00  1.84960938e+00  1.45800781e+00
  1.06347656e+00  6.03027344e-01 -2.79541016e-02 -6.36718750e-01
 -1.10449219e+00 -1.51757812e+00 -1.98144531e+00 -2.38476562e+00
 -2.56640625e+00 -2.60156250e+00 -2.71289062e+00 -2.86718750e+00
 -2.92578125e+00 -2.84765625e+00 -2.66210938e+00 -2.39843750e+00
 -2.09375000e+00 -1.80175781e+00 -1.51757812e+00 -1.26464844e+00
 -1.04785156e+00 -8.82324219e-01 -7.37792969e-01 -6.10351562e-01
 -5.18554688e-01 -3.97949219e-01 -2.66601562e-01 -1.69189453e-01
 -8.44116211e-02 -2.04772949e-02  1.94091797e-02  9.97924805e-03
 -2.60009766e-02 -8.55712891e-02 -1.52832031e-01 -2.06054688e-01
 -2.52685547e-01 -2.54150391e-01 -2.59033203e-01 -2.72949219e-01
 -2.58056641e-01 -2.36328125e-01 -1.94824219e-01 -1.30371094e-01
 -7.75146484e-02 -2.89306641e-02 -1.2

class= 42 shape(new5)= (518, 486)	mx= 3.615234375	mn= -2.935546875

4250 = 518
